### Comments:
- Work in progress
- To use cuda I need to call .cuda() on every tensor I instantiate during my train() procedure 
- Separate data for training and testing
- In this demo the dataloader uses batch_size = 1 given that the .csv with the data has a single 64x64x64 volume

### 08/2019
- why I cant import Variable when its called within a function????

In [1]:
from models.REDCNN import REDCNN
from utils import utils

import torch
import torch.nn as nn
from torch.autograd import Variable

In [1]:
#wanna know how long does it take to train the net

from datetime import datetime
from dateutil.relativedelta import relativedelta

def time_diff(t_a, t_b):
    t_diff = relativedelta(t_b, t_a) 
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [ ]:
#hyperparameters:
num_epochs = 1
batch_size = 1
learning_rate = 0.001

In [6]:
###########################################

#helper function that should be in utils
def var_or_cuda(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x) 

###########################################

In [4]:
#instantiating the model:

model = REDCNN()
model.double()

#loss function
criterion = nn.BCEWithLogitsLoss()

#optimizer algorithm
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

#dataset, dataloader
csv_file = r'C:\Users\Juanig\Desktop\Desktop_\ct images\para probar el procesamiento de los volumenes\100_FBPPhil_500FBP.csv'
dataset = utils.CTVolumesDataset(csv_file)
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

#if gpu available
if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()

In [29]:
enu = enumerate(dataloader)
len(dataloader)

1

In [7]:
def var_or_cuda(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x) 

#set the model to train
model.train()
total_step = len(dataloader)

#train

start = datetime.now()

for epoch in range(num_epochs):
    for i, (lo_res, hi_res) in enumerate(dataloader):
        
        #add an extra dimension:
        #find out why the following line does not work
        #lo_res = utils.var_or_cuda( lo_res.unsqueeze(1) )
        lo_res = Variable(lo_res.unsqueeze(1))
        #print(lo_res.size())
        if lo_res.size()[0] != batch_size:
            print("batch_size != {} drop last incompatible batch".format( batch_size ))
            continue
        
        
        #forward pass       
        outputs = model(lo_res)
        loss = criterion(outputs, hi_res.unsqueeze(1))
        
        #backward & optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
         
        if (i+1) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
               
stop = datetime.now()
dur = time_diff(start, stop)

        
torch.save(model.state_dict(), '3d_autoencoder.pkl')  

torch.Size([1, 1, 64, 64, 64])
Epoch [1/1], Step [1/1], Loss: 0.7738
